In [1]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch
import pandas as pd
from datasets import load_dataset
import os
from PIL import Image
from transformers.image_utils import load_image

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
model = AutoModelForVision2Seq.from_pretrained("HuggingFaceTB/SmolVLM-Instruct",
                                                # torch_dtype=torch.bfloat16,
                                                _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager").to(DEVICE)


tokenizer_config.json:   0%|          | 0.00/4.48k [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


In [2]:
df = pd.read_csv('data_VLM.csv')
df

,Unnamed: 0,url,cache,path
0,0,http://www.grandchambery.fr/fileadmin/mediathe...,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/0.png
1,1,http://www.ville-saint-ay.fr/userfile/fichier-...,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/1.png
2,2,https://www.gatine-racan.fr/wp-content/uploads...,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/2.png
3,3,https://www.ville-mazeres.fr/IMG/pdf/2023_1_1.pdf,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/3.png
4,4,https://www.fier-et-usses.com/cms_viewFile.php...,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/4.png
...,...,...,...,...
495,495,https://www.estuaire-sillon.fr/fileadmin/media...,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/495.png
496,496,https://plombieres-les-dijon.fr/wp-content/upl...,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/496.png
497,497,https://www.orne.gouv.fr/contenu/telechargemen...,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/497.png
498,498,https://www.vosges.gouv.fr/contenu/telechargem...,https://datapolitics-public.s3.gra.io.cloud.ov...,extracted_images/498.png


In [ ]:
def label_data(data, device = 'mps'):
    """
    Input:
        data : pd.DataFrame, a DataFrame object containing the column 'path' with paths to images
    Output:
        pd.DataFrame, a DataFrame object with a new added column 'predicted_date'
    """
    hypothesis_dates = []
    model.to(device)
    for i, row in data.iterrows():
        path = row['path']
        if pd.isna(path): 
            hypothesis_dates.append(None)
        else:
            img = load_image(path)
            messages = [
                    {
                        "role": "user",
                        "content": [
                            {"type": "image"},
                            {"type": "text", "text": """
                                            What is the publication date of this document? Answer in a numerical date format YYYY-MM-DD. """}
                        ]
                    },
                ]
            
            prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
            inputs = processor(text=prompt, images=[img], return_tensors="pt")
            inputs.to(device)
            generated_ids = model.generate(**inputs, max_new_tokens=50)
            
            generated_texts = processor.batch_decode(
                generated_ids,
                skip_special_tokens=True,
            )
            
            hyp_text = generated_texts[0].split('Assistant:')[-1]
            hypothesis_dates.append(hyp_text)
            
    data['predicted_date'] = hypothesis_dates
    return data

In [24]:
df_labelled = label_data(df)

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/transformers/pytorch_utils.py:325: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


In [20]:
pd.isna((df['path'][6]))

True

In [24]:
df_labelled.to_csv('data_VLM_labeled.csv')

In [4]:
df_labelled = pd.read_csv('data_VLM_labeled.csv')
# df_labelled['predicted_date'] = df_labelled['df_labelled']

In [29]:
import datetime
from dateparser import parse

In [30]:
def format_date(datestr):
    if pd.isna(datestr) : return None
    date_formatted = parse(datestr)
    if date_formatted is not None: 
        date_formatted=date_formatted.strftime('%d/%m/%Y')
    return date_formatted

In [31]:
df_labelled['formatted_date'] = df_labelled['predicted_date'].apply(format_date)

In [25]:
df_new_labels = pd.read_csv('dataset_text_url.csv')[['Gold published date', 'url']]
df_labelled = df_labelled.merge(df_new_labels, on='url')

In [16]:
df_labelled['is_correct'] = df_labelled['formatted_date'] == df_labelled['Gold published date']
df_labelled['is_correct_month_year'] = df_labelled['formatted_date'].str[3:] == df_labelled['Gold published date'].str[3:]

In [ ]:
pd.read_csv('dataset_text_url.csv')[['url','Gold published date']]

,url,Gold published date
0,http://www.ville-saint-ay.fr/userfile/fichier-...,16/01/2023
1,https://www.gatine-racan.fr/wp-content/uploads...,25/01/2023
2,https://www.ville-mazeres.fr/IMG/pdf/2023_1_1.pdf,02/02/2023
3,https://www.fier-et-usses.com/cms_viewFile.php...,26/01/2023
4,https://www.grandbourg.fr/cms_viewFile.php?idt...,16/01/2023
...,...,...
495,https://plombieres-les-dijon.fr/wp-content/upl...,02/04/2024
496,https://www.orne.gouv.fr/contenu/telechargemen...,09/01/2024
497,https://www.vosges.gouv.fr/contenu/telechargem...,22/11/2022
498,http://www.grandchambery.fr/fileadmin/mediathe...,21/12/2023


## Accuracy with rows where empty image paths are dropped

In [17]:
df_labelled[['Gold published date', 'is_correct']].dropna()['is_correct'].mean()

0.34826883910386963

In [18]:
df_labelled[['Gold published date', 'is_correct_month_year']].dropna()['is_correct_month_year'].mean()

0.5132382892057027